In [2]:
import pandas as pd
import numpy as np


In [4]:
# Load the data
data = pd.read_csv('complete_games_table.csv')

data.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,2023-24,1610612738,BOS,Boston Celtics,42300405,17/06/2024,BOS vs. DAL,W,240,106,...,0.850,15,36,51,25,9,2,7,15,18.0
1,2023-24,1610612742,DAL,Dallas Mavericks,42300405,17/06/2024,DAL @ BOS,L,240,88,...,0.538,7,28,35,18,4,4,13,20,-18.0
2,2023-24,1610612742,DAL,Dallas Mavericks,42300404,14/06/2024,DAL vs. BOS,W,240,122,...,0.682,13,39,52,21,7,2,8,17,38.0
3,2023-24,1610612738,BOS,Boston Celtics,42300403,12/06/2024,BOS @ DAL,W,240,106,...,0.929,6,30,36,26,4,6,9,19,7.0
4,2023-24,1610612742,DAL,Dallas Mavericks,42300403,12/06/2024,DAL vs. BOS,L,240,99,...,0.875,7,36,43,15,5,1,8,17,-7.0


In [5]:
data.describe

<bound method NDFrame.describe of       SEASON_ID     TEAM_ID TEAM_ABBREVIATION            TEAM_NAME   GAME_ID  \
0       2023-24  1610612738               BOS       Boston Celtics  42300405   
1       2023-24  1610612742               DAL     Dallas Mavericks  42300405   
2       2023-24  1610612742               DAL     Dallas Mavericks  42300404   
3       2023-24  1610612738               BOS       Boston Celtics  42300403   
4       2023-24  1610612742               DAL     Dallas Mavericks  42300403   
...         ...         ...               ...                  ...       ...   
50674   2000-01  1610612759               SAS    San Antonio Spurs  20000009   
50675   2000-01  1610612760               SEA  Seattle SuperSonics  20000013   
50676   2000-01  1610612761               TOR      Toronto Raptors  20000005   
50677   2000-01  1610612764               WAS   Washington Wizards  20000003   
50678   2000-01  1610612765               DET      Detroit Pistons  20000005   

     

In [ ]:
# Select relevant columns
data = data[['TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_DATE', 'MATCHUP',
             'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 
             'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 
             'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS']]

In [4]:
# Fixing the MATCHUP column to split HOME_TEAM and AWAY_TEAM
data['HOME_TEAM'] = np.where(data['MATCHUP'].str.contains(' vs. '),
                             data['MATCHUP'].str.extract(r'^([^vs.]+)')[0],
                             data['MATCHUP'].str.extract(r'@ (.+)$')[0])

data['AWAY_TEAM'] = np.where(data['MATCHUP'].str.contains(' vs. '),
                             data['MATCHUP'].str.extract(r'vs. (.+)$')[0],
                             data['MATCHUP'].str.extract(r'^([^@]+)')[0])

# Removing international and non-NBA teams
non_nba_teams = ['ADL', 'FLA', 'MAC', 'MEL', 'MRA', 'NZB', 'PER', 'RMD', 'SLA', 
                 'CHN', 'FBU', 'MAL', 'MMT', 'MOS', 'MTA', 'RMA', 'BAU', 'BNE', 
                 'EPT', 'FEN', 'GUA', 'LAB', 'LRY', 'OLP', 'PAN', 'SDS', 'SYD', 
                 'ZAK']

data = data[~data['HOME_TEAM'].isin(non_nba_teams)]
data = data[~data['AWAY_TEAM'].isin(non_nba_teams)]

# Removing spaces in team abbreviations
data['AWAY_TEAM'] = data['AWAY_TEAM'].str.replace(' ', '')
data['HOME_TEAM'] = data['HOME_TEAM'].str.replace(' ', '')

# Standardizing team abbreviations
team_abbreviation_corrections = {
    'NJN': 'BKN', 'NOH': 'NOP', 'NOK': 'NOP', 'VAN': 'MEM', 'SEA': 'OKC', 'CHH': 'CHA'
}

data['TEAM_ABBREVIATION'] = data['TEAM_ABBREVIATION'].replace(team_abbreviation_corrections)
data['HOME_TEAM'] = data['HOME_TEAM'].replace(team_abbreviation_corrections)
data['AWAY_TEAM'] = data['AWAY_TEAM'].replace(team_abbreviation_corrections)


In [9]:
# Initial Feature Enginnering:
# Adding the following columns:

# 1. Points per minute:
data['PPM'] = round(data['PTS'] / 48, 2)

# 2. Estimated Possessions:
data['POSSESSIONS'] = data['FGA'] - data['OREB'] + data['STL']
# 3. Points per possession:
data['PTSperP'] = round(data['PTS'] / data['POSSESSIONS'], 3)

# 4. Effective Field Goal Percentage:
data['EFGPCT'] = round((data['FGM'] + 0.5 * data['FG3M']) / data['FGA'] * 100, 3)

# 5. Pace:
data['PACE'] = round((data['POSSESSIONS'] / 48), 3)

# 6. Home or Away:
data['HOME'] = np.where(data['MATCHUP'].str.contains(' vs. '), 1, 0)

# 7. Scoring Opportunities
data['Scoring_Opportunities'] = data['FGA'] + data['FG3A'] + data['FTA']

# 8. Defensive Index
data['Defensive_Index'] = data['DREB'] + data['STL'] + data['BLK']

# 9. Turnover to Assist Ratio
data['TO_to_AST_Ratio'] = data['TOV'] / (data['AST'] + 1e-10)




In [10]:
data.head()

# Save the data for use for analysis:
data.to_csv('finalised_NBA_Data.csv', index=False)

In [11]:
data.head()

,TEAM_ABBREVIATION,TEAM_NAME,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,...,AWAY_TEAM,PPM,POSSESSIONS,PTSperP,EFGPCT,PACE,HOME,Scoring_Opportunities,Defensive_Index,TO_to_AST_Ratio
0,BOS,Boston Celtics,17/06/2024,BOS vs. DAL,W,240,106,38,89,0.427,...,DAL,2.21,89.0,1.191,50.000,89.0,1,148,47,0.280000
1,DAL,Dallas Mavericks,17/06/2024,DAL @ BOS,L,240,88,35,78,0.449,...,DAL,1.83,89.2,0.987,51.923,89.2,0,128,36,0.722222
2,DAL,Dallas Mavericks,14/06/2024,DAL vs. BOS,W,240,122,46,91,0.505,...,BOS,2.54,94.8,1.287,58.791,94.8,1,150,48,0.380952
3,BOS,Boston Celtics,12/06/2024,BOS @ DAL,W,240,106,38,82,0.463,...,BOS,2.21,90.6,1.170,56.707,90.6,0,142,40,0.346154
4,DAL,Dallas Mavericks,12/06/2024,DAL vs. BOS,L,240,99,38,86,0.442,...,BOS,2.06,93.4,1.060,49.419,93.4,1,127,42,0.533333
